## 1

In [1]:
import xarray as xr
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Air temperature/*.nc'

t2m=xr.open_mfdataset(path)

In [3]:
t2m

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    air        (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [6]:
new_lat = np.arange(90, -91, -2.5).astype(np.float32)
new_lon = np.arange(0, 360, 2.5).astype(np.float32)

# Resample to the new grid size
resized_t2m = t2m.interp(lat=new_lat, lon=new_lon)
resized_t2m.air

<xarray.DataArray 'air' (time: 15340, lat: 73, lon: 144)>
dask.array<chunked_aware_interpnd, shape=(15340, 73, 144), dtype=float32, chunksize=(366, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:     mean Daily Air temperature at 2 m
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NCEP Reanalysis Daily Averages
    level_desc:    2 m
    statistic:     Mean
    parent_stat:   Individual Obs
    actual_range:  [177.68 316.55]
    valid_range:   [150. 400.]

In [5]:
resized_t2m.air

<xarray.DataArray 'air' (time: 15340, lat: 73, lon: 144)>
dask.array<chunked_aware_interpnd, shape=(15340, 73, 144), dtype=float32, chunksize=(366, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:     mean Daily Air temperature at 2 m
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NCEP Reanalysis Daily Averages
    level_desc:    2 m
    statistic:     Mean
    parent_stat:   Individual Obs
    actual_range:  [177.68 316.55]
    valid_range:   [150. 400.]

In [6]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)



# Use the sel method to select data within the specified Bangladesh region
data_bangladesh_resized = resized_t2m.sel(lat=lat_range,lon=lon_range)





In [7]:
data_bangladesh_resized.lon

<xarray.DataArray 'lon' (lon: 2)>
array([90. , 92.5], dtype=float32)
Coordinates:
  * lon      (lon) float32 90.0 92.5
Attributes:
    units:          degrees_east
    long_name:      Longitude
    actual_range:   [  0.    358.125]
    standard_name:  longitude
    axis:           X

In [8]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
t2m_df = pd.DataFrame(columns=['time', 'lat', 'lon', 't2m'])

# Calculate the number of chunks
num_chunks = len(data_bangladesh_resized.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = data_bangladesh_resized.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    t2m = chunk.air.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
#         'lat': np.tile(lat, len(time) * len(lon)),
#         'lon': np.tile(lon, len(time) * len(lat)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        
        
        't2m': t2m.ravel()
    })

    t2m_df = t2m_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
t2m_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,t2m
0,1981-01-01,25.0,90.0,286.047974
1,1981-01-01,22.5,92.5,280.861481
2,1981-01-01,25.0,90.0,292.600800
3,1981-01-01,22.5,92.5,285.080719
4,1981-01-02,25.0,90.0,287.016754
...,...,...,...,...
61355,2022-12-30,22.5,92.5,285.860657
61356,2022-12-31,25.0,90.0,285.811737
61357,2022-12-31,22.5,92.5,285.012299
61358,2022-12-31,25.0,90.0,292.700378


In [9]:
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm



df = pd.DataFrame(columns=["time", "lat", "lon", "t2m"])

# Loop through dimensions and populate the DataFrame
for time_val in tqdm(data.coords["time"].values[:5],desc='Main Loop',leave=True):
    for lat_val in tqdm(data.coords["lat"].values,desc=f"loop for time_val={time_val}",leave=False):
        for lon_val in tqdm(data.coords["lon"].values,desc=f"loop for lon_val={lat_val}",leave=False):
            t2m = data.sel(time=time_val, lat=lat_val, lon=lon_val).air.values
            df = df.append({"time": time_val, "lat": lat_val, "lon": lon_val, "t2m": t2m}, ignore_index=True)

,time,lat,lon,t2m
0,1981-01-01,25.0,90.0,286.047974
1,1981-01-01,22.5,92.5,280.861481
2,1981-01-01,25.0,90.0,292.600800
3,1981-01-01,22.5,92.5,285.080719
4,1981-01-02,25.0,90.0,287.016754
5,1981-01-02,22.5,92.5,283.308136
6,1981-01-02,25.0,90.0,293.262115
7,1981-01-02,22.5,92.5,286.289581
8,1981-01-03,25.0,90.0,285.222900
9,1981-01-03,22.5,92.5,284.111389


In [10]:
# t2m_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/t2m_df.csv')

# 2

# Maximum Temperature

In [11]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Maximum Temperature//*.nc'

data_tmax=xr.open_mfdataset(path)

In [12]:
data_tmax

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    tmax       (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

# Resizing grid size

In [13]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
new_ds = data_tmax.interp(lat=new_lat, lon=new_lon)

In [14]:
new_ds

<xarray.Dataset>
Dimensions:    (time: 15340, lat: 73, lon: 144, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: nbnds
Data variables:
    tmax       (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

# Extracting data

In [15]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)

# Use the sel method to select data within the specified Bangladesh region
tmax_bangladesh_resized = new_ds.sel(lat=lat_range,lon=lon_range)

In [16]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
tmax_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'tmax'])

# Calculate the number of chunks
num_chunks = len(tmax_bangladesh_resized.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = tmax_bangladesh_resized.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    tmax = chunk.tmax.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'tmax': tmax.ravel()
    })

    tmax_df = tmax_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
tmax_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,tmax
0,1981-01-01,25.0,90.0,293.807465
1,1981-01-01,22.5,92.5,291.438507
2,1981-01-01,25.0,90.0,296.828430
3,1981-01-01,22.5,92.5,294.985016
4,1981-01-02,25.0,90.0,294.823517
...,...,...,...,...
61355,2022-12-30,22.5,92.5,297.287415
61356,2022-12-31,25.0,90.0,295.736115
61357,2022-12-31,22.5,92.5,295.295074
61358,2022-12-31,25.0,90.0,298.031860


In [17]:
# tmax_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/tmax_df.csv')

# 3
# Minimum Temperature


In [18]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Minimum Temperature///*.nc'

data_tmin=xr.open_mfdataset(path)

In [19]:
data_tmin.tmin

<xarray.DataArray 'tmin' (time: 15340, lat: 94, lon: 192)>
dask.array<concatenate, shape=(15340, 94, 192), dtype=float32, chunksize=(366, 94, 192), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Attributes:
    long_name:     Daily Minimum Temperature at 2 m
    units:         degK
    precision:     2
    GRIB_id:       16
    GRIB_name:     T MIN
    var_desc:      Minimum Temperature
    dataset:       NCEP Reanalysis Daily Values
    level_desc:    2 m
    statistic:     Daily Minimum
    parent_stat:   Individual Obs
    actual_range:  [169.20001 309.2    ]
    valid_range:   [150. 400.]

In [20]:
# Resizing grid size

In [21]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
new_ds = data_tmin.interp(lat=new_lat, lon=new_lon)

In [22]:
new_ds

<xarray.Dataset>
Dimensions:    (time: 15340, lat: 73, lon: 144, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: nbnds
Data variables:
    tmin       (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

# Extracting data

In [23]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)



# Use the sel method to select data within the specified Bangladesh region
tmin_bangladesh_resized = new_ds.sel(lat=lat_range,lon=lon_range)





In [24]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
tmin_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'tmin'])

# Calculate the number of chunks
num_chunks = len(tmin_bangladesh_resized.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = tmin_bangladesh_resized.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    tmin = chunk.tmin.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'tmin': tmin.ravel()
    })

    tmin_df = tmin_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
tmin_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,tmin
0,1981-01-01,25.0,90.0,280.111206
1,1981-01-01,22.5,92.5,272.609222
2,1981-01-01,25.0,90.0,288.973724
3,1981-01-01,22.5,92.5,277.570099
4,1981-01-02,25.0,90.0,279.398926
...,...,...,...,...
61355,2022-12-30,22.5,92.5,275.808075
61356,2022-12-31,25.0,90.0,276.473755
61357,2022-12-31,22.5,92.5,275.029541
61358,2022-12-31,25.0,90.0,288.641602


In [25]:
# tmin_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/tmin_df.csv')

## 4

In [26]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Surface Pressure-NCAR/*.nc'
surface_pressure=xr.open_mfdataset(path)

In [27]:
surface_pressure

<xarray.Dataset>
Dimensions:    (lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    pres       (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [28]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)


# Use the sel method to select data within the specified Bangladesh region
surface_pressure_bangladesh = surface_pressure.sel(lat=lat_range,lon=lon_range)

In [29]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
surface_pressure_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'Surface Pressure'])

# Calculate the number of chunks
num_chunks = len(surface_pressure_bangladesh.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = surface_pressure_bangladesh.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    pres = chunk.pres.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'Surface Pressure': pres.ravel()
    })

    surface_pressure_df = surface_pressure_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
surface_pressure_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,Surface Pressure
0,1981-01-01,25.0,90.0,100540.0
1,1981-01-01,22.5,92.5,96700.0
2,1981-01-01,25.0,90.0,101940.0
3,1981-01-01,22.5,92.5,95940.0
4,1981-01-02,25.0,90.0,100470.0
...,...,...,...,...
61355,2022-12-30,22.5,92.5,96225.0
61356,2022-12-31,25.0,90.0,100815.0
61357,2022-12-31,22.5,92.5,96885.0
61358,2022-12-31,25.0,90.0,102212.5


In [30]:
# surface_pressure_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/surface_pressure_df.csv')

# 5

In [31]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Sea level pressure/*.nc'
slp=xr.open_mfdataset(path)


In [32]:
slp

<xarray.Dataset>
Dimensions:    (lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    slp        (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [33]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)


# Use the sel method to select data within the specified Bangladesh region
bangladesh_slp = slp.sel(lat=lat_range,lon=lon_range)

In [34]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
slp_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'slp'])

# Calculate the number of chunks
num_chunks = len(bangladesh_slp.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_slp.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    slp = chunk.slp.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'slp': slp.ravel()
    })

    slp_df = slp_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
slp_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,slp
0,1981-01-01,25.0,90.0,101615.0
1,1981-01-01,22.5,92.5,101680.0
2,1981-01-01,25.0,90.0,101453.0
3,1981-01-01,22.5,92.5,101483.0
4,1981-01-02,25.0,90.0,101550.0
...,...,...,...,...
61355,2022-12-30,22.5,92.5,101720.0
61356,2022-12-31,25.0,90.0,101907.5
61357,2022-12-31,22.5,92.5,101807.5
61358,2022-12-31,25.0,90.0,101722.5


In [35]:
# slp_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/slp_df.csv')

# 6

In [36]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Precipitation/*.nc'
precp_data=xr.open_mfdataset(path)


In [37]:
precp_data

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    prate      (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [38]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east

# Resample to the new grid size
resized_precp = precp_data.interp(lat=new_lat, lon=new_lon)

In [39]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)




# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_precp = resized_precp.sel(lat=lat_range,lon=lon_range)
bangladesh_resized_precp

<xarray.Dataset>
Dimensions:    (time: 15340, lat: 2, lon: 2, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat        (lat) float32 25.0 22.5
  * lon        (lon) float32 90.0 92.5
Dimensions without coordinates: nbnds
Data variables:
    prate      (time, lat, lon) float32 dask.array<chunksize=(365, 2, 2), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [40]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
prate_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'prate'])

# Calculate the number of chunks
num_chunks = len(bangladesh_resized_precp.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_resized_precp.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    prate = chunk.prate.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'prate': prate.ravel()
    })

    prate_df = prate_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
prate_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,prate
0,1981-01-01,25.0,90.0,2.675627e-06
1,1981-01-01,22.5,92.5,4.526225e-06
2,1981-01-01,25.0,90.0,1.624216e-07
3,1981-01-01,22.5,92.5,1.415758e-07
4,1981-01-02,25.0,90.0,6.726953e-06
...,...,...,...,...
61355,2022-12-30,22.5,92.5,0.000000e+00
61356,2022-12-31,25.0,90.0,1.094083e-06
61357,2022-12-31,22.5,92.5,2.083968e-07
61358,2022-12-31,25.0,90.0,0.000000e+00


In [41]:
import pandas as pd

precp_df=prate_df

precp_df['daily_precp'] = precp_df['prate'] * 86400


precp_df


,time,lat,lon,prate,daily_precp
0,1981-01-01,25.0,90.0,2.675627e-06,0.231174
1,1981-01-01,22.5,92.5,4.526225e-06,0.391066
2,1981-01-01,25.0,90.0,1.624216e-07,0.014033
3,1981-01-01,22.5,92.5,1.415758e-07,0.012232
4,1981-01-02,25.0,90.0,6.726953e-06,0.581209
...,...,...,...,...,...
61355,2022-12-30,22.5,92.5,0.000000e+00,0.000000
61356,2022-12-31,25.0,90.0,1.094083e-06,0.094529
61357,2022-12-31,22.5,92.5,2.083968e-07,0.018005
61358,2022-12-31,25.0,90.0,0.000000e+00,0.000000


In [42]:
# precp_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/precp_df.csv')

# 7

In [43]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/10m u/*.nc'
u_wind=xr.open_mfdataset(path)


In [44]:
u_wind

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    uwnd       (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [45]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
resized_uwind = u_wind.interp(lat=new_lat, lon=new_lon)

In [46]:
resized_uwind

<xarray.Dataset>
Dimensions:    (time: 15340, lat: 73, lon: 144, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: nbnds
Data variables:
    uwnd       (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [47]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)




# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_uwind = resized_uwind.sel(lat=lat_range,lon=lon_range)

In [48]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
uwind_df = pd.DataFrame(columns=['time', 'lat', 'lon', '10m_uwind'])

# Calculate the number of chunks
num_chunks = len(bangladesh_resized_uwind.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_resized_uwind.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    uwind = chunk.uwnd.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        '10m_uwind': uwind.ravel()
    })

    uwind_df = uwind_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
uwind_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,10m_uwind
0,1981-01-01,25.0,90.0,-0.939862
1,1981-01-01,22.5,92.5,-0.572421
2,1981-01-01,25.0,90.0,-2.372857
3,1981-01-01,22.5,92.5,-0.787714
4,1981-01-02,25.0,90.0,-0.938654
...,...,...,...,...
61355,2022-12-30,22.5,92.5,0.724410
61356,2022-12-31,25.0,90.0,0.112595
61357,2022-12-31,22.5,92.5,0.021775
61358,2022-12-31,25.0,90.0,-0.440561


In [49]:
# uwind_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/uwind_df.csv')

# 8

In [50]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/10m v/*.nc'
v_wind=xr.open_mfdataset(path)

In [51]:
v_wind

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    vwnd       (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [52]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
resized_vwind = v_wind.interp(lat=new_lat, lon=new_lon)

In [53]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)



# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_vwind = resized_vwind.sel(lat=lat_range,lon=lon_range)

In [54]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
vwind_df = pd.DataFrame(columns=['time', 'lat', 'lon', '10m_vwind'])

# Calculate the number of chunks
num_chunks = len(bangladesh_resized_vwind.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_resized_vwind.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    vwind = chunk.vwnd.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        '10m_vwind': vwind.ravel()
    })

    vwind_df = vwind_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
vwind_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,10m_vwind
0,1981-01-01,25.0,90.0,-0.149908
1,1981-01-01,22.5,92.5,-0.593328
2,1981-01-01,25.0,90.0,-2.195786
3,1981-01-01,22.5,92.5,-1.198803
4,1981-01-02,25.0,90.0,-0.102309
...,...,...,...,...
61355,2022-12-30,22.5,92.5,-0.445329
61356,2022-12-31,25.0,90.0,0.450114
61357,2022-12-31,22.5,92.5,-0.524987
61358,2022-12-31,25.0,90.0,0.349927


In [55]:
# import pandas as pd

# # Assuming df_uwind and df_vwind are your dataframes
# # Merge based on 'time', 'lat', and 'lon'
# wind_df = pd.merge(uwind_df, vwind_df, on=['time', 'lat', 'lon'])

# # If you want to merge based on an index, you can use left_index=True and right_index=True
# # merged_df = pd.merge(df_uwind, df_vwind, left_index=True, right_index=True, suffixes=('_uwind', '_vwind'))


In [56]:
# wind_df

In [57]:
# vwind_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/vwind_df.csv')

# 9

In [58]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Geopotential Height/*.nc'
geo_hgt=xr.open_mfdataset(path)


In [59]:
geo_hgt

<xarray.Dataset>
Dimensions:    (level: 17, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    hgt        (time, level, lat, lon) float32 dask.array<chunksize=(365, 17, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC Reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [60]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)




desired_levels=[1000,850,500,300]
# Use the sel method to select data within the specified Bangladesh region
bangladesh_geo_hgt = geo_hgt.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [61]:
bangladesh_geo_hgt

<xarray.Dataset>
Dimensions:    (level: 4, lat: 2, lon: 2, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 850.0 500.0 300.0
  * lat        (lat) float32 25.0 22.5
  * lon        (lon) float32 90.0 92.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    hgt        (time, level, lat, lon) float32 dask.array<chunksize=(365, 4, 2, 2), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC Reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [62]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
geo_hgt_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'geo_hgt_1000hpa','geo_hgt_850hpa','geo_hgt_500hpa','geo_hgt_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_geo_hgt.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_geo_hgt.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    geo_hgt_1000hpa = chunk.sel(level=1000).hgt.values
    geo_hgt_850hpa = chunk.sel(level=850).hgt.values
    geo_hgt_500hpa = chunk.sel(level=500).hgt.values
    geo_hgt_300hpa = chunk.sel(level=300).hgt.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'geo_hgt_1000hpa': geo_hgt_1000hpa.ravel(),
        'geo_hgt_850hpa': geo_hgt_850hpa.ravel(),
        'geo_hgt_500hpa': geo_hgt_500hpa.ravel(),
        'geo_hgt_300hpa': geo_hgt_300hpa.ravel()
    })

    geo_hgt_df = geo_hgt_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
geo_hgt_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,geo_hgt_1000hpa,geo_hgt_850hpa,geo_hgt_500hpa,geo_hgt_300hpa
0,1981-01-01,25.0,90.0,125.00,1491.75,5739.25,9509.50
1,1981-01-01,22.5,92.5,132.25,1491.50,5733.00,9500.75
2,1981-01-01,25.0,90.0,131.25,1504.50,5775.50,9575.00
3,1981-01-01,22.5,92.5,134.50,1501.75,5771.25,9571.00
4,1981-01-02,25.0,90.0,121.50,1491.50,5737.00,9492.50
...,...,...,...,...,...,...,...
61355,2022-12-30,22.5,92.5,157.00,1532.25,5839.25,9664.00
61356,2022-12-31,25.0,90.0,161.75,1530.25,5766.25,9549.00
61357,2022-12-31,22.5,92.5,164.00,1531.00,5759.50,9551.50
61358,2022-12-31,25.0,90.0,154.75,1531.00,5815.75,9636.25


In [63]:
# geo_hgt_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/geo_hgt_df.csv')

# 10

In [64]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Relative Humidity/*.nc'
rhum=xr.open_mfdataset(path)


In [65]:
rhum

<xarray.Dataset>
Dimensions:    (level: 8, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    rhum       (time, level, lat, lon) float32 dask.array<chunksize=(365, 8, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [66]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)


desired_levels=[1000,850,500,300]
# desired_levels=[300]

# Use the sel method to select data within the specified Bangladesh region
bangladesh_rhum = rhum.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [67]:
bangladesh_rhum

<xarray.Dataset>
Dimensions:    (level: 4, lat: 2, lon: 2, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 850.0 500.0 300.0
  * lat        (lat) float32 25.0 22.5
  * lon        (lon) float32 90.0 92.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    rhum       (time, level, lat, lon) float32 dask.array<chunksize=(365, 4, 2, 2), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [68]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
rhum_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'rhum_1000hpa','rhum_850hpa','rhum_500hpa','rhum_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_rhum.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_rhum.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    rhum_1000hpa = chunk.sel(level=1000).rhum.values
    rhum_850hpa = chunk.sel(level=850).rhum.values
    rhum_500hpa = chunk.sel(level=500).rhum.values
    rhum_300hpa = chunk.sel(level=300).rhum.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'rhum_1000hpa': rhum_1000hpa.ravel(),
        'rhum_850hpa': rhum_850hpa.ravel(),
        'rhum_500hpa': rhum_500hpa.ravel(),
        'rhum_300hpa': rhum_300hpa.ravel()
    })

    rhum_df = rhum_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
rhum_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,rhum_1000hpa,rhum_850hpa,rhum_500hpa,rhum_300hpa
0,1981-01-01,25.0,90.0,76.500015,74.000015,6.50,11.25
1,1981-01-01,22.5,92.5,78.250015,77.500015,5.25,9.25
2,1981-01-01,25.0,90.0,57.000015,46.000008,7.75,5.25
3,1981-01-01,22.5,92.5,62.500008,49.000008,6.75,1.50
4,1981-01-02,25.0,90.0,74.500015,61.500015,11.25,9.75
...,...,...,...,...,...,...,...
61355,2022-12-30,22.5,92.5,43.750000,28.000000,11.75,11.25
61356,2022-12-31,25.0,90.0,62.500000,56.500000,7.00,17.00
61357,2022-12-31,22.5,92.5,60.750000,63.000000,7.25,15.00
61358,2022-12-31,25.0,90.0,45.500000,36.250000,4.50,13.75


In [69]:
# rhum_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/rhum_df.csv')

# 11

In [70]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Air temperature multiple levels/*.nc'
air_temp=xr.open_mfdataset(path)


In [71]:
air_temp

<xarray.Dataset>
Dimensions:    (level: 17, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    air        (time, level, lat, lon) float32 dask.array<chunksize=(365, 17, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [72]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)



desired_levels=[1000,850,500,300]
# desired_levels=[300]

# Use the sel method to select data within the specified Bangladesh region
bangladesh_air_temp = air_temp.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [73]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
air_temp_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'air_temp_1000hpa','air_temp_850hpa','air_temp_500hpa','air_temp_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_air_temp.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_air_temp.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    air_temp_1000hpa = chunk.sel(level=1000).air.values
    air_temp_850hpa = chunk.sel(level=850).air.values
    air_temp_500hpa = chunk.sel(level=500).air.values
    air_temp_300hpa = chunk.sel(level=300).air.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'air_temp_1000hpa': air_temp_1000hpa.ravel(),
        'air_temp_850hpa': air_temp_850hpa.ravel(),
        'air_temp_500hpa': air_temp_500hpa.ravel(),
        'air_temp_300hpa': air_temp_300hpa.ravel()
    })

    air_temp_df = air_temp_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
air_temp_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,air_temp_1000hpa,air_temp_850hpa,air_temp_500hpa,air_temp_300hpa
0,1981-01-01,25.0,90.0,290.125000,283.150024,262.400024,241.675003
1,1981-01-01,22.5,92.5,287.449982,282.300018,262.125000,241.725006
2,1981-01-01,25.0,90.0,290.875000,283.750000,263.975006,242.925018
3,1981-01-01,22.5,92.5,288.599976,283.600006,264.250000,242.975006
4,1981-01-02,25.0,90.0,291.225006,282.825012,261.825012,240.875000
...,...,...,...,...,...,...,...
61355,2022-12-30,22.5,92.5,291.500000,283.750000,267.524963,243.674988
61356,2022-12-31,25.0,90.0,291.349976,282.124969,263.474976,242.424988
61357,2022-12-31,22.5,92.5,290.375000,282.274963,263.125000,243.500000
61358,2022-12-31,25.0,90.0,292.599976,284.324982,266.824982,243.349976


In [74]:
# air_temp_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/air_temp_df.csv')

# 12

In [75]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Specific Humidity Multiple levels/*.nc'
shum=xr.open_mfdataset(path)


In [76]:
shum

<xarray.Dataset>
Dimensions:    (level: 8, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    shum       (time, level, lat, lon) float32 dask.array<chunksize=(365, 8, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/06/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [77]:
# lat_range = slice(29,18.0)  # take lat values in backwards directions.
# lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
lat_range = slice(26.7,20.7) 
lon_range = slice(88,92.7)
# lat_range = slice(27.5,20) 
# lon_range = slice(87.5,95)



desired_levels=[1000,850,500,300]
# desired_levels=[300]

# Use the sel method to select data within the specified Bangladesh region
bangladesh_shum = shum.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [78]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
shum_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'shum_1000hpa','shum_850hpa','shum_500hpa','shum_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_shum.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_shum.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    shum_1000hpa = chunk.sel(level=1000).shum.values
    shum_850hpa = chunk.sel(level=850).shum.values
    shum_500hpa = chunk.sel(level=500).shum.values
    shum_300hpa = chunk.sel(level=300).shum.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'shum_1000hpa': shum_1000hpa.ravel(),
        'shum_850hpa': shum_850hpa.ravel(),
        'shum_500hpa': shum_500hpa.ravel(),
        'shum_300hpa': shum_300hpa.ravel()
    })

    shum_df = shum_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
shum_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,shum_1000hpa,shum_850hpa,shum_500hpa,shum_300hpa
0,1981-01-01,25.0,90.0,0.009118,0.006580,0.000222,0.000119
1,1981-01-01,22.5,92.5,0.007985,0.006525,0.000172,0.000087
2,1981-01-01,25.0,90.0,0.007218,0.004200,0.000279,0.000052
3,1981-01-01,22.5,92.5,0.006265,0.004450,0.000263,0.000065
4,1981-01-02,25.0,90.0,0.009428,0.005455,0.000372,0.000080
...,...,...,...,...,...,...,...
61355,2022-12-30,22.5,92.5,0.005520,0.002625,0.000588,0.000129
61356,2022-12-31,25.0,90.0,0.008135,0.004732,0.000258,0.000170
61357,2022-12-31,22.5,92.5,0.007277,0.005328,0.000258,0.000165
61358,2022-12-31,25.0,90.0,0.006417,0.003503,0.000217,0.000151


In [79]:
# shum_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/shum_df.csv')

In [80]:
# sanjid

## Merge all dfs

In [ ]:
# dfs=[t2m_df,tmax_df,tmin_df,surface_pressure_df,slp_df,precp_df,uwind_df, vwind_df,geo_hgt_df,rhum_df,air_temp_df,shum_df]
# # Initialize the merged_df with the first DataFrame in the list
# merged_df = dfs[0]

# # Loop through the remaining DataFrames and merge them one by one
# for df in dfs[1:]:
#     merged_df = pd.merge(merged_df, df)

# # Now merged_df contains the merged result
# merged_df

In [ ]:
# sanjid

In [ ]:
# from functools import reduce

# dfs=[t2m_df,tmax_df,tmin_df,surface_pressure_df,slp_df,precp_df,uwind_df, vwind_df,geo_hgt_df,rhum_df,air_temp_df,shum_df]
# merge_func = lambda left, right: pd.merge(left, right, on=['time', 'lat', 'lon'])
# merged_df = reduce(merge_func, dfs)

In [1]:
import pandas as pd

t2m_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/t2m_df.csv')
tmax_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/tmax_df.csv')
tmin_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/tmin_df.csv')
surface_pressure_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/surface_pressure_df.csv')
slp_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/slp_df.csv')
precp_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/precp_df.csv')
uwind_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/uwind_df.csv')
vwind_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/vwind_df.csv')
geo_hgt_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/geo_hgt_df.csv')
rhum_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/rhum_df.csv')
air_temp_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/air_temp_df.csv')
shum_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/shum_df.csv')


In [2]:
dfs=[t2m_df,tmax_df,tmin_df,surface_pressure_df,slp_df,precp_df,uwind_df, vwind_df,geo_hgt_df,rhum_df,air_temp_df,shum_df]
# Initialize the merged_df with the first DataFrame in the list
merged_df = dfs[0]

# Loop through the remaining DataFrames and merge them one by one
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df)

# Now merged_df contains the merged result
merged_df

,Unnamed: 0,time,lat,lon,t2m,tmax,tmin,Surface Pressure,slp,prate,...,rhum_500hpa,rhum_300hpa,air_temp_1000hpa,air_temp_850hpa,air_temp_500hpa,air_temp_300hpa,shum_1000hpa,shum_850hpa,shum_500hpa,shum_300hpa
0,0,1981-01-01,25.0,90.0,286.04797,293.80746,280.11120,100540.0,101615.0,2.675628e-06,...,6.50,11.25,290.12500,283.15002,262.40002,241.67500,0.009118,0.006580,0.000222,0.000119
1,1,1981-01-01,22.5,92.5,280.86148,291.43850,272.60922,96700.0,101680.0,4.526225e-06,...,5.25,9.25,287.44998,282.30002,262.12500,241.72500,0.007985,0.006525,0.000172,0.000087
2,2,1981-01-01,25.0,90.0,292.60080,296.82843,288.97372,101940.0,101453.0,1.624216e-07,...,7.75,5.25,290.87500,283.75000,263.97500,242.92502,0.007218,0.004200,0.000279,0.000052
3,3,1981-01-01,22.5,92.5,285.08072,294.98502,277.57010,95940.0,101483.0,1.415758e-07,...,6.75,1.50,288.59998,283.60000,264.25000,242.97500,0.006265,0.004450,0.000263,0.000065
4,4,1981-01-02,25.0,90.0,287.01675,294.82352,279.39893,100470.0,101550.0,6.726953e-06,...,11.25,9.75,291.22500,282.82500,261.82500,240.87500,0.009428,0.005455,0.000372,0.000080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61355,61355,2022-12-30,22.5,92.5,285.86066,297.28740,275.80807,96225.0,101720.0,0.000000e+00,...,11.75,11.25,291.50000,283.75000,267.52496,243.67499,0.005520,0.002625,0.000588,0.000129
61356,61356,2022-12-31,25.0,90.0,285.81174,295.73610,276.47375,100815.0,101907.5,1.094083e-06,...,7.00,17.00,291.34998,282.12497,263.47498,242.42499,0.008135,0.004732,0.000258,0.000170
61357,61357,2022-12-31,22.5,92.5,285.01230,295.29507,275.02954,96885.0,101807.5,2.083968e-07,...,7.25,15.00,290.37500,282.27496,263.12500,243.50000,0.007278,0.005328,0.000258,0.000166
61358,61358,2022-12-31,25.0,90.0,292.70038,298.03186,288.64160,102212.5,101722.5,0.000000e+00,...,4.50,13.75,292.59998,284.32498,266.82498,243.34998,0.006417,0.003502,0.000217,0.000151


In [ ]:
merged_df1 = pd.merge(t2m_df, tmax_df)
merged_df2 = pd.merge(merged_df1, tmin_df)
merged_df3 = pd.merge(merged_df2, surface_pressure_df)
merged_df4 = pd.merge(merged_df3, slp_df)
merged_df5 = pd.merge(merged_df4, precp_df)
merged_df6 = pd.merge(merged_df5, uwind_df)
merged_df7 = pd.merge(merged_df6, vwind_df)
merged_df8 = pd.merge(merged_df7, geo_hgt_df)
merged_df9 = pd.merge(merged_df8, rhum_df)
merged_df10 = pd.merge(merged_df9, air_temp_df)
merged_df11 = pd.merge(merged_df10, shum_df)
merged_df11

# merged_df12 = pd.merge(t2m_df, tmax_df)



In [ ]:
merged_df1 = pd.merge(t2m_df, tmax_df)
merged_df1

In [ ]:
merged_df2 = pd.merge(merged_df1, tmin_df)
merged_df2

In [ ]:
merged_df3 = pd.merge(merged_df2, surface_pressure_df)
merged_df3

In [ ]:
merged_df4 = pd.merge(merged_df3, slp_df)
merged_df4

In [ ]:
merged_df5 = pd.merge(merged_df4, precp_df)
merged_df5

In [ ]:
merged_df6 = pd.merge(merged_df5, uwind_df)
merged_df6

In [ ]:
merged_df7 = pd.merge(merged_df6, vwind_df)
merged_df7

In [ ]:
merged_df8 = pd.merge(merged_df7, geo_hgt_df)
merged_df8

In [ ]:
merged_df9 = pd.merge(merged_df8, rhum_df)
merged_df9

In [ ]:
merged_df10 = pd.merge(merged_df9, air_temp_df)
merged_df10

In [ ]:
merged_df11 = pd.merge(merged_df10, shum_df)
merged_df11

In [ ]:
dfs=[t2m_df,tmax_df,tmin_df,surface_pressure_df,slp_df,precp_df,uwind_df, vwind_df,geo_hgt_df,rhum_df,air_temp_df,shum_df]
# dfs[0]

In [ ]:
# Initialize the merged_df with the first DataFrame in the list
merged_df = dfs[0]

# Loop through the remaining DataFrames and merge them one by one
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df)

# Now merged_df contains the merged result


In [ ]:
sanjid

In [ ]:
from functools import reduce

dfs=[t2m_df,tmax_df,tmin_df,surface_pressure_df,slp_df,precp_df]
merge_func = lambda left, right: pd.merge(left, right, on=['time', 'lat', 'lon'])
df1 = reduce(merge_func, dfs)


In [ ]:
df1.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/df1.csv')

In [ ]:
from functools import reduce

dfs=[slp_df,precp_df,uwind_df, vwind_df]
merge_func = lambda left, right: pd.merge(left, right, on=['time', 'lat', 'lon'])
df2 = reduce(merge_func, dfs)

In [ ]:
df2.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/df2.csv')

In [ ]:
from functools import reduce

dfs=[geo_hgt_df,rhum_df,air_temp_df,shum_df]
merge_func = lambda left, right: pd.merge(left, right, on=['time', 'lat', 'lon'])
df3 = reduce(merge_func, dfs)

In [ ]:
df3.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/df3.csv')

In [ ]:
from functools import reduce

dfs=[df1,df2,df3]
merge_func = lambda left, right: pd.merge(left, right, on=['time', 'lat', 'lon'])
final_data= reduce(merge_func, dfs)

In [ ]:
# import pandas as pd

# df1=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/geo_hgt_df.csv')
# df2=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/df2.csv')
# df3=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/df3.csv')

# df1
# lat = [27.5, 25. , 22.5, 20. ]
# lon = [85. , 87.5, 90. , 92.5, 95. ]

dfs=['t2m_df','tmax_df','tmin_df','surface_pressure_df','slp_df','precp_df','uwind_df', 'vwind_df','geo_hgt_df','rhum_df','air_temp_df','shum_df']

# dfs[1]

for i in dfs:
    print(f"df1=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/{i}.csv')")





# from functools import reduce

# dfs=[df1,df2]
# merge_func = lambda left, right: pd.merge(left, right, on=['time', 'lat', 'lon'])
# final_data= reduce(merge_func, dfs)






In [ ]:
import pandas as pd



t2m_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/t2m_df.csv')
tmax_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/tmax_df.csv')
tmin_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/tmin_df.csv')
surface_pressure_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/surface_pressure_df.csv')
slp_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/slp_df.csv')
precp_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/precp_df.csv')
uwind_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/uwind_df.csv')
vwind_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/vwind_df.csv')
geo_hgt_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/geo_hgt_df.csv')
rhum_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/rhum_df.csv')
air_temp_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/air_temp_df.csv')
shum_df=pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/indv df v1/shum_df.csv')


In [ ]:
dfs=[t2m_df,tmax_df,tmin_df,surface_pressure_df,slp_df,precp_df,uwind_df, vwind_df,geo_hgt_df,rhum_df,air_temp_df,shum_df]


# # Initialize the merged_df with the first DataFrame in the list
# merged_df = dfs[0]

# # Loop through the remaining DataFrames and merge them one by one
# for df in dfs[1:]:
#     merged_df = pd.merge(merged_df, df, on=['time', 'lat', 'lon'])

merged_df = pd.merge(t2m_df, tmax_df)

In [ ]:
merged_df

In [ ]:
merged_df

In [ ]:
merged_df.to_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/testdf v3.csv')

In [ ]:
sanjid

In [ ]:
import glob
import pandas as pd
from functools import reduce


import dask.dataframe as dd
from functools import reduce

# Assuming all your CSV files are in the same directory
file_paths = glob.glob('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/*.csv')
file_paths


In [ ]:
from functools import reduce


import pandas as pd

# Read the DataFrames from the CSV files
left_df = pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/air_temp_df.csv',index_col=0)
right_df = pd.read_csv('/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Preprocessed Data/geo_hgt_df.csv',index_col=0)

# Merge the DataFrames on the specified columns
merged_df = pd.merge(left_df, right_df, on=['time', 'lat', 'lon'])



In [ ]:
merged_df